In [1]:
import duckdb
import os
import pandas as pd
import re

from duckdb.typing import *

## Qulaity assessment

In [2]:
input_root_dir = '../data/'
output_root_dir = '../clean_data/'

input_files_dirs = [os.path.join(input_root_dir,'2023')]
output_files_dirs = [os.path.join(output_root_dir,'2023')]

# input_files_dirs = [os.path.join(input_root_dir,x) for x in os.listdir(input_root_dir)]
# output_files_dirs = [os.path.join(output_root_dir,x) for x in os.listdir(input_root_dir)]

In [3]:
metadata_headers = ['region','federative_unit','state','code','latitude','longitude','altitude','foundation_date']
column_names = ['date','time','total_precipitation','avg_atmospheric_pressure','max_atmospheric_pressure','min_atmospheric_pressure','global_radiation','avg_air_temperature','dew_point','max_temperature','min_temperature','max_dew_point','min_dew_point','max_relative_air_humidity','min_relative_air_humidity','relative_air_humidity','wind_direction','max_wind_gust','wind_speed']
# column_types = [VARCHAR,VARCHAR,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE,DOUBLE]
# column_names = ['column00','column01','column02','column03','column04','column05','column06','column07','column08','column09','column10','column11','column12','column13','column14','column15','column16','column17','column18']


In [4]:
df = pd.read_csv('../data/2022/INMET_CO_DF_A045_AGUAS EMENDADAS_01-01-2022_A_31-12-2022.CSV',
                 encoding='ISO-8859-1',
                 delimiter=';',
                 skiprows=9,
                 decimal=',',
                 names=column_names,
                 usecols=column_names, # We have to specify `usecols, because the files have a trailing ;, which causes an extra column to be created`
                 na_values=-9999,)

df.drop_duplicates(subset=['date', 'time'])


,date,time,total_precipitation,avg_atmospheric_pressure,max_atmospheric_pressure,min_atmospheric_pressure,global_radiation,avg_air_temperature,dew_point,max_temperature,min_temperature,max_dew_point,min_dew_point,max_relative_air_humidity,min_relative_air_humidity,relative_air_humidity,wind_direction,max_wind_gust,wind_speed
0,2022/01/01,0000 UTC,0.0,897.1,897.1,896.3,NaN,19.7,18.4,19.8,19.6,18.4,18.1,92.0,90.0,92.0,24.0,4.1,2.0
1,2022/01/01,0100 UTC,0.0,897.8,897.9,897.2,NaN,19.6,18.0,19.8,19.5,18.4,18.0,92.0,90.0,90.0,21.0,4.3,2.0
2,2022/01/01,0200 UTC,0.0,897.7,897.9,897.7,NaN,18.9,17.6,19.6,18.9,18.0,17.6,92.0,90.0,92.0,31.0,3.9,1.7
3,2022/01/01,0300 UTC,0.0,897.3,897.7,897.3,NaN,18.6,17.5,18.9,18.6,17.6,17.4,93.0,92.0,93.0,25.0,4.2,2.1
4,2022/01/01,0400 UTC,0.0,896.7,897.3,896.7,NaN,18.8,17.7,18.8,18.6,17.7,17.5,94.0,93.0,93.0,26.0,4.2,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022/12/31,1900 UTC,0.0,898.7,899.7,898.6,1088.3,25.7,18.6,25.7,25.2,20.6,18.4,75.0,64.0,65.0,98.0,3.3,1.0
8756,2022/12/31,2000 UTC,0.0,898.5,898.9,898.5,1001.6,25.8,18.1,26.4,25.6,19.0,17.2,67.0,57.0,63.0,46.0,3.0,0.8
8757,2022/12/31,2100 UTC,0.0,899.3,899.3,898.5,334.9,23.8,19.2,26.1,23.8,19.3,17.2,75.0,60.0,75.0,5.0,2.8,0.7
8758,2022/12/31,2200 UTC,0.0,899.8,899.8,899.2,43.4,21.3,18.1,23.8,21.3,19.0,17.0,84.0,70.0,82.0,160.0,1.7,0.4


In [5]:
def parse_time(time:str) -> str:
    if re.match('^([01][\d]|2[0-3])([0-5][\d]) UTC$',time) is not None:
        time = time[:2] + ':' + time[2:4]
    
    return time+':00'


def parse_date(date:str) -> str:
    if re.match('^[\d]{4}/[\d]{2}/[\d]{2}$', date):
        return date.replace('/','-')
    if re.match('^[\d]{2}/[\d]{2}/[\d]{2}$', date):
        date = date.split('/')
        date[0], date[-1] = '20'+date[-1], date[0]
        return '-'.join(date)
    
    return date


In [6]:
try:
    duckdb.create_function('parse_time', parse_time, [VARCHAR], VARCHAR)
except duckdb.NotImplementedException:
    pass

try:
    duckdb.create_function('parse_date', parse_date, [VARCHAR], VARCHAR)
except duckdb.NotImplementedException:
    pass

In [7]:
duckdb.sql(f"SELECT parse_time(time) AS time,COLUMNS(* EXCLUDE time) FROM df WHERE COLUMNS(* EXCLUDE (date,time)) IS NULL ORDER BY date, time")

┌──────────┬────────────┬─────────────────────┬───┬──────────────────────┬────────────────┬───────────────┬────────────┐
│   time   │    date    │ total_precipitation │ … │ relative_air_humid…  │ wind_direction │ max_wind_gust │ wind_speed │
│ varchar  │  varchar   │       double        │   │        double        │     double     │    double     │   double   │
├──────────┼────────────┼─────────────────────┼───┼──────────────────────┼────────────────┼───────────────┼────────────┤
│ 23:00:00 │ 2022/12/08 │                NULL │ … │                 NULL │           NULL │          NULL │       NULL │
│ 00:00:00 │ 2022/12/09 │                NULL │ … │                 NULL │           NULL │          NULL │       NULL │
│ 01:00:00 │ 2022/12/09 │                NULL │ … │                 NULL │           NULL │          NULL │       NULL │
│ 02:00:00 │ 2022/12/09 │                NULL │ … │                 NULL │           NULL │          NULL │       NULL │
│ 01:00:00 │ 2022/12/10 │       

In [8]:
df = duckdb.sql("SELECT * FROM df EXCEPT SELECT * FROM df WHERE COLUMNS(* EXCLUDE (date,time)) IS NULL ORDER BY date, time").to_df()

In [9]:
df = duckdb.sql("""
           SELECT parse_date(date)::date AS date,
            parse_time(time)::time AS time,
           COLUMNS(* EXCLUDE (wind_direction,max_relative_air_humidity,min_relative_air_humidity,relative_air_humidity,time,date,wind_speed)),
           max_relative_air_humidity::UTINYINT AS max_relative_air_humidity,
           min_relative_air_humidity::UTINYINT AS min_relative_air_humidity,
           relative_air_humidity::UTINYINT AS relative_air_humidity,
           wind_direction::USMALLINT AS wind_direction,
           wind_speed
           FROM df""").to_df()

In [10]:
df

,date,time,total_precipitation,avg_atmospheric_pressure,max_atmospheric_pressure,min_atmospheric_pressure,global_radiation,avg_air_temperature,dew_point,max_temperature,min_temperature,max_dew_point,min_dew_point,max_wind_gust,max_relative_air_humidity,min_relative_air_humidity,relative_air_humidity,wind_direction,wind_speed
0,2022-01-01,00:00:00,0.0,897.1,897.1,896.3,NaN,19.7,18.4,19.8,19.6,18.4,18.1,4.1,92.0,90.0,92,24.0,2.0
1,2022-01-01,01:00:00,0.0,897.8,897.9,897.2,NaN,19.6,18.0,19.8,19.5,18.4,18.0,4.3,92.0,90.0,90,21.0,2.0
2,2022-01-01,02:00:00,0.0,897.7,897.9,897.7,NaN,18.9,17.6,19.6,18.9,18.0,17.6,3.9,92.0,90.0,92,31.0,1.7
3,2022-01-01,03:00:00,0.0,897.3,897.7,897.3,NaN,18.6,17.5,18.9,18.6,17.6,17.4,4.2,93.0,92.0,93,25.0,2.1
4,2022-01-01,04:00:00,0.0,896.7,897.3,896.7,NaN,18.8,17.7,18.8,18.6,17.7,17.5,4.2,94.0,93.0,93,26.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8669,2022-12-31,19:00:00,0.0,898.7,899.7,898.6,1088.3,25.7,18.6,25.7,25.2,20.6,18.4,3.3,75.0,64.0,65,98.0,1.0
8670,2022-12-31,20:00:00,0.0,898.5,898.9,898.5,1001.6,25.8,18.1,26.4,25.6,19.0,17.2,3.0,67.0,57.0,63,46.0,0.8
8671,2022-12-31,21:00:00,0.0,899.3,899.3,898.5,334.9,23.8,19.2,26.1,23.8,19.3,17.2,2.8,75.0,60.0,75,5.0,0.7
8672,2022-12-31,22:00:00,0.0,899.8,899.8,899.2,43.4,21.3,18.1,23.8,21.3,19.0,17.0,1.7,84.0,70.0,82,160.0,0.4


In [14]:
df['wind_direction'].count()

7736